In [1]:
from pyscipopt import Model, Eventhdlr, quicksum, SCIP_EVENTTYPE,SCIP_HEURTIMING
import pyscipopt
import sys
from tqdm import tqdm
import random
import pandas as pd

In [2]:
from pyscipopt import Model, Eventhdlr, SCIP_EVENTTYPE

class FixedVarsAtNode(Eventhdlr):
    """PySCIPOpt Event handler to write fixed vars of each node to a text file."""

    def __init__(self,random_seed):
        
        #define the path of file
        # self.output_var_info_file = "var_info/var_info.txt"
        
        #others
        self.df_y_info = pd.DataFrame(columns=['Before', 'After', 'y_label'])

        self.count=0
        self.count_limit=5
        
        #make random seed
        self.random_seed = random_seed
        self.random = random.Random(random_seed)

    def eventexec(self, event):
        if self.count<self.count_limit:
            
            # 创建一个空的DataFrame
            df_var_info = pd.DataFrame(columns=['Var', 'Type', 'LowerBound','UpperBound','GLB','GUB'])
            
            
            fixedvars=[]
            # get node number
            nodenumber = event.getNode().getNumber()
        
            #use_heuristic
            use_heuristic = self.random.choice([True, False])
            if use_heuristic:
                #     # 保存未使用启发式时的目标函数值
                # obj_value_no_heuristic = self.model.getObjVal() 
                #写入node信息
                self.transvars = self.model.getVars(transformed=True)
                for var in self.transvars:
                    var_name = var.name
                    Glb = var.getUbGlobal()#获取变量的全局上界
                    Gub = var.getLbGlobal()#获取变量的全局下界
                    lb = var.getLbLocal()  # 获取变量的局部下界
                    ub = var.getUbLocal()  # 获取变量的局部上界
                    var_type = var.vtype()
                    
                    var_info_add_df ={'Var': var_name, 'Type': var_type, 'LowerBound': lb, 'UpperBound': ub, 'GLB':Glb, 'GUB':Gub}
                    #append to df_var_info
                    df_var_info = df_var_info.append(var_info_add_df, ignore_index=True)
                
                #输出为csv文件
                file_name='var_info/'+str(self.count) +"var_info.csv"
                df_var_info.to_csv(file_name, index=False) 
                
               
                # obj_value_heuristic = self.model.getObjVal() 
                # if_y_better = self.check_if_better(obj_value_no_heuristic,obj_value_heuristic)
                #每个node运行后就直接存入对应的行，最后全运完直接把文件导出
                y_info_add_df ={'Before': obj_value_no_heuristic, 'After': obj_value_heuristic, 'y_label': if_y_better}
                self.df_y_info.append(y_info_add_df, ignore_index=True)
            else: 
                pass
                
            self.count+=1
        else:
             pass

        
    def check_if_better(self,before,after):
        obj_fun=self.model.getObjectiveSense()
        
        if obj_fun == 'minimize':
            if before>after:
                if_better = 1
            else:
                if_better = 0
        else:
            if before < after:
                if_better = 1
            else:
                if_better = 0
        return if_better
            
        
    def write_y_info_file(self , info):
        #输出为csv文件
        file_name = str(self.count) +"_y_info.csv"
        self.df_y_info.to_csv(file_name, index=False)
        

    def eventinit(self):
        self.model.catchEvent(SCIP_EVENTTYPE.NODESOLVED, self)

    def eventexit(self):
        self.model.dropEvent(SCIP_EVENTTYPE.NODESOLVED, self)


In [7]:
if __name__ == "__main__":
    # Specify the output file path
    #output_var_info_file = "var_info.txt"

    
    test = pyscipopt.scip.Model()
    test.readProblem("/Users/oukeikou/Desktop/sunruoyao/easy-sample/gen-ip002.mps")
    print("read done")


    # Create and add event handler with the specified output file
    eventhdlr = FixedVarsAtNode(100)
    test.includeEventhdlr(eventhdlr, "FixedVarsAtNode", "Python event handler to write fixed variables after each solved node")

    # Optimize the problem
    test.optimize()
    print("optimized")
    
    eventhdlr.write_y_info_file()

    # ... 获取最优解等其他操作 ...
    if test.getStatus() == "optimal":
        # 获取最优解的变量值
        optimal_solution = {var.name: test.getVal(var) for var in test.getVars()}

        # 输出最优解
        print("Optimal Solution:")
        for var_name, var_value in optimal_solution.items():
            print(f"{var_name}: {var_value}")
        node_count = test.getNNodes()
        print(node_count)

    else:
            print("No optimal solution found~")

    # Free resources
    del test


original problem has 41 variables (0 bin, 41 int, 0 impl, 0 cont) and 24 constraints
read done


NameError: name 'FixedVarsAtNode' is not defined

In [23]:
# 启用Rins启发式
model.includeHeur(
    rins_heur,
    name="RINS",
    desc="RINS Heuristic",
    dispchar="R",
    timingmask=SCIP_HEURTIMING.BEFORENODE
)

In [14]:
import random
import pyscipopt as scip

# 创建SCIP Solver
model = scip.Model()

# 设置随机种子
random_seed = 10
random = random.Random(random_seed)
use_heuristic = random.choice([True, False])

# 定义一个简单的线性规划问题
x = model.addVar(vtype="B", name="x")
y = model.addVar(vtype="B", name="y")
model.addCons(x + 2 * y <= 1, "c1")
model.addCons(2 * x + y <= 1, "c2")
model.setObjective(x + y, "minimize")

# 主循环
node_count = 0
# node_objective_values = []


while True:
    # 优化模型
    model.optimize()

    # 收集目标函数值
    node_objective_values.append(model.getObjVal())
    
    # Check if the current node is the first or second node
    if node_count == 0 or node_count == 1:
        model.setIntParam("heuristics/rins/freq",0.5)
        # model.includeHeur(rins_heuristic, "Rins", "Rins heuristic", "Y", timingmask=scip.SCIP_HEURTIMING.BEFORENODE)
    else:
        # Remove the Rins heuristic for subsequent nodes
        model.setIntParam("heuristics/rins/freq",1)

    # 继续搜索下一个节点
    node_count += 1

# 打印每个节点的目标函数值
for i, obj_value in enumerate(node_objective_values):
    print(f"Node {i + 1}: Objective Value = {obj_value}")

# 关闭SCIP Solver
model.freeProb()

# 打印总的节点数量
print(f"Total Nodes: {node_count}")

feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
presolving (1 rounds: 1 fast, 0 medium, 0 exhaustive):
 2 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
transformed 1/1 original solutions to the transformed problem space
Presolving Time: 0.00

SCIP Status        : problem is solved [optimal solution found]
Solving Time (sec) : 0.00
Solving Nodes      : 0
Primal Bound       : +0.00000000000000e+00 (1 solutions)
Dual Bound         : +0.00000000000000e+00
Gap                : 0.00 %


KeyboardInterrupt: 

In [5]:
rins_heur = scip.Heur()

In [4]:
model.includeNodeCallback(node_callback, "MyNodeCallback")

AttributeError: 'pyscipopt.scip.Model' object has no attribute 'includeNodeCallback'

In [9]:
import pyscipopt as scip

# 创建SCIP Solver
model = scip.Model() 

# 定义一个简单的线性规划问题
x = model.addVar(vtype="B", name="x")
y = model.addVar(vtype="B", name="y")
model.addCons(x + 2 * y <= 1, "c1")
model.addCons(2 * x + y <= 1, "c2")
model.setObjective(x + y, "minimize")


# 创建节点回调函数
def node_callback(scip_model, node_data):
    if node_data.getDepth() == 3:  # 例如，只在深度为3的节点上运行启发式
        rins_heur.execute(scip_model)

# 注册节点回调函数
model.includeNodeRelaxEvent(node_callback, "MyNodeCallback", "Node Callback Description")

# 主循环
model.optimize()

# 获取最优解
x_value = model.getVal(x)
y_value = model.getVal(y)
optimal_obj_value = model.getObjVal()

# 打印结果
print(f"x = {x_value}")
print(f"y = {y_value}")
print(f"Optimal Objective Value = {optimal_obj_value}")

# 关闭SCIP Solver
model.freeProb()


AttributeError: 'pyscipopt.scip.Model' object has no attribute 'includeNodeRelaxEvent'

In [16]:
import pyscipopt as scip

# 创建SCIP Solver
model = scip.Model()

# 定义一个简单的线性规划问题
x = model.addVar(vtype="B", name="x")
y = model.addVar(vtype="B", name="y")
model.addCons(x + 2 * y <= 1, "c1")
model.addCons(2 * x + y <= 1, "c2")
model.setObjective(x + y, "minimize")

# 定义节点回调函数
def node_callback(scip_model, node_data):
    current_node = node_data.getNode()  # 获取当前节点
    depth = node_data.getDepth()  # 获取当前节点的深度
    
    # 在特定深度的节点上执行节点计算
    if depth == 3:
        # 在这里执行自定义的节点计算操作
        print(f"Performing node computation at depth {depth}")

# 创建节点回调对象
node_callback_obj = model.createNodeCallback(callback=node_callback)

# 注册节点回调对象
model.includeObjEvent(node_callback_obj, "NodeCallbackName", "NodeCallbackDescription")

# 主循环
model.optimize()

# 关闭SCIP Solver
model.freeProb()


AttributeError: 'pyscipopt.scip.Model' object has no attribute 'createNodeCallback'

In [6]:
import gc
import weakref

import pytest

from pyscipopt import Model, Heur, SCIP_RESULT, SCIP_PARAMSETTING, SCIP_HEURTIMING
from pyscipopt.scip import is_memory_freed

from util import is_optimized_mode

class MyHeur(Heur):
            return {"result": SCIP_RESULT.DIDNOTFIND}

def test_heur():
    assert round(sol[y]) == 0.0

def test_heur_memory():
    if is_optimized_mode():
       pytest.skip()

    def inner():
        s = Model()
        heuristic = MyHeur()
        s.includeHeur(heuristic, "PyHeur", "custom heuristic implemented in python", "Y", timingmask=SCIP_HEURTIMING.BEFORENODE)
        return weakref.proxy(heuristic)

    heur_prox = inner()
    gc.collect() # necessary?
    with pytest.raises(ReferenceError):
        heur_prox.name

    assert is_memory_freed()

if __name__ == "__main__":
    test_heur()

ModuleNotFoundError: No module named 'pytest'

In [8]:
# List all names (methods, classes, attributes) in the pyscipopt module
all_names = dir(pyscipopt)

# Print the list of names
for name in all_names:
    print(name)

Benders
Benderscut
Branchrule
Conshdlr
Eventhdlr
Expr
Heur
LP
Model
Multidict
Nodesel
Presol
Pricer
Prop
Reader
SCIP_BENDERSENFOTYPE
SCIP_BRANCHDIR
SCIP_EVENTTYPE
SCIP_HEURTIMING
SCIP_LPSOLSTAT
SCIP_PARAMEMPHASIS
SCIP_PARAMSETTING
SCIP_PRESOLTIMING
SCIP_PROPTIMING
SCIP_RESULT
SCIP_ROWORIGINTYPE
SCIP_STAGE
SCIP_STATUS
Sepa
__builtins__
__cached__
__doc__
__file__
__loader__
__name__
__package__
__path__
__spec__
__version__
cos
exp
log
multidict
os
quickprod
quicksum
scip
sin
sqrt
